<a href="https://colab.research.google.com/github/hadisan13/Dicoding/blob/main/Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tweet-preprocessor

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
nltk.download('stopwords')

import os


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

Device: grpc://10.100.33.130:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.100.33.130:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.100.33.130:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Number of replicas: 8


In [ ]:
train_df = pd.read_csv(r'/content/Corona_NLP_train.csv', encoding='latin_1')
test_df = pd.read_csv(r'/content/Corona_NLP_test.csv', encoding='latin_1')

df = pd.concat([train_df, test_df], ignore_index=True)

df

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
...,...,...,...,...,...,...
44950,3794,48746,Israel ??,16-03-2020,Meanwhile In A Supermarket in Israel -- People...,Positive
44951,3795,48747,"Farmington, NM",16-03-2020,Did you panic buy a lot of non-perishable item...,Negative
44952,3796,48748,"Haverford, PA",16-03-2020,Asst Prof of Economics @cconces was on @NBCPhi...,Neutral
44953,3797,48749,NaN,16-03-2020,Gov need to do somethings instead of biar je r...,Extremely Negative


In [ ]:
coret = ['UserName', 'ScreenName', 'Location', 'TweetAt']
df.drop(columns=coret, inplace=True)

In [ ]:
def change_sen(sentiment):
    if sentiment == "Extremely Positive":
        return 'positive'
    elif sentiment == "Extremely Negative":
        return 'negative'
    elif sentiment == "Positive":
        return 'positive'
    elif sentiment == "Negative":
        return 'negative'
    else:
        return 'netural'

In [ ]:
X_raw = df['OriginalTweet']
Y_raw = df['Sentiment']

Y_raw = Y_raw.apply(change_sen)

category = pd.get_dummies(Y_raw)
Y = pd.concat([Y_raw, category], axis=1)
Y.drop(columns = ['Sentiment'], inplace=True)

In [ ]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
def clean(text):
    # specific
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can\'t", "can not", text)

    # general
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    # remove url
    #         text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    text = re.sub(r'http\S+', '', text)
    # remove @
    text = re.sub(r'@\w+', '', text)
    # remove #
    text = re.sub(r'#\w+', '', text)
    
    text = re.sub(r'<.*?>', '', text)
        
    text = text.split()
        
    text = ' '.join([word.lower() for word in text if word.lower() not in stop])
        
    return text

In [ ]:
X = X_raw.apply(clean)

In [ ]:
from sklearn.model_selection import train_test_split

X_trainval, X_test, y_trainval, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, random_state=42)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

word_index = tokenizer.word_index

max_len = max(X_train.apply(len))

sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_val = tokenizer.texts_to_sequences(X_val) 

X_train_padded = pad_sequences(sequences_train, max_len, padding='post')
X_val_padded = pad_sequences(sequences_val, max_len, padding='post')


In [ ]:
# Modelling starts with GloVe pre-trained 100
embedding_dict={}
with open('/content/glove.6B.100d.txt','r',encoding='utf-8') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        embedding_dict[word]=vectors
f.close()

In [ ]:
num_words=len(word_index)+1
embedding_matrix=np.zeros((num_words,100))

for word,i in word_index.items():
    if i > num_words:
        print("Inside i greater than")
        continue
    
    emb_vec=embedding_dict.get(word)
    if emb_vec is not None:
        #print("i =" , i)
        embedding_matrix[i]=emb_vec

In [ ]:
embedding_layer = tf.keras.layers.Embedding(len(word_index) + 1,
                                            100,
                                            weights=[embedding_matrix],
                                            input_length=max_len,
                                            trainable=False)

In [ ]:
def My_Model():

  result = tf.keras.Sequential([
    embedding_layer,
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True, recurrent_dropout=0.4)),
    tf.keras.layers.GlobalMaxPool1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(3, activation='softmax')
  ])

  return result

In [ ]:
with strategy.scope():
  model = My_Model()

model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 286, 100)          3126000   
_________________________________________________________________
bidirectional_16 (Bidirectio (None, 286, 256)          234496    
_________________________________________________________________
global_max_pooling1d_11 (Glo (None, 256)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 64)                16448     
_________________________________________________________________
dropout_13 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 3)                 195       
Total params: 3,377,139
Trainable params: 251,139
Non-trainable params: 3,126,000
_____________________________________

In [ ]:
with strategy.scope():
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train_padded, y_train, epochs = 20, validation_data = (X_val_padded, y_val), batch_size = 64, shuffle=True)

Epoch 1/20
450/450 [==============================] - 36s 65ms/step - loss: 0.8736 - accuracy: 0.5944 - val_loss: 0.6619 - val_accuracy: 0.7272
Epoch 2/20
450/450 [==============================] - 25s 55ms/step - loss: 0.6459 - accuracy: 0.7404 - val_loss: 0.5800 - val_accuracy: 0.7670
Epoch 3/20
450/450 [==============================] - 25s 55ms/step - loss: 0.5490 - accuracy: 0.7881 - val_loss: 0.5436 - val_accuracy: 0.7833
Epoch 4/20
450/450 [==============================] - 25s 56ms/step - loss: 0.4727 - accuracy: 0.8274 - val_loss: 0.5179 - val_accuracy: 0.8023
Epoch 5/20
450/450 [==============================] - 25s 56ms/step - loss: 0.4133 - accuracy: 0.8536 - val_loss: 0.5298 - val_accuracy: 0.7984
Epoch 6/20
450/450 [==============================] - 25s 56ms/step - loss: 0.3510 - accuracy: 0.8791 - val_loss: 0.4938 - val_accuracy: 0.8172
Epoch 7/20
450/450 [==============================] - 25s 56ms/step - loss: 0.2884 - accuracy: 0.9035 - val_loss: 0.5153 - val_accuracy: